In [1]:
import torch
from datasets import load_dataset

DATASET_NAME = "DiscoResearch/germanrag"

dataset = load_dataset(DATASET_NAME, split="train")

In [2]:
print(dataset[0].keys())

dict_keys(['contexts', 'question', 'answer', 'positive_ctx_idx'])


In [4]:
from dataclasses import dataclass

@dataclass
class DataEntry:
    context: str
    question: str
    answer: str
    
d = DataEntry("a", "b", "c")
d.__dict__

{'context': 'a', 'question': 'b', 'answer': 'c'}

In [24]:
def prepare_jsonl_from_data_instance(data_instance):
    contexts = data_instance["contexts"]
    question = data_instance["question"]
    answer = data_instance["answer"]
    pos_ctx_idx = data_instance["positive_ctx_idx"]
    pos_ctx= contexts[pos_ctx_idx]
    
    
    new_entries = list()
    
    for ctx in contexts:
        if ctx == pos_ctx:
            new_entry = DataEntry(ctx, question, answer)
        else:
            # no answer is possible
            msg = "Bei dem gegebenen Kontext ist keine Antwort möglich."
            new_entry = DataEntry(ctx, question, msg)
        
        new_entries.append(new_entry.__dict__)
            
    return new_entries

In [6]:
from src.docs import EasyNegativeFinder

finder = EasyNegativeFinder("sentence-transformers/distiluse-base-multilingual-cased-v2", "cuda", "./persisted_data")

2024-02-28 21:40:12.898 | INFO     | src.docs:__load_embedding_model:18 - Initialising Embeddings with model ::sentence-transformers/distiluse-base-multilingual-cased-v2
2024-02-28 21:40:17.154 | SUCCESS  | src.docs:__load_embedding_model:23 - Done


In [7]:
data_instance = dataset[0]

In [11]:
from tqdm.auto import tqdm

unique_contexts = set()
for _, d in tqdm(enumerate(dataset)):
    for ctx in d["contexts"]:
        unique_contexts.add(ctx)

0it [00:00, ?it/s]

In [12]:
finder.init_vector_store(list(unique_contexts))

2024-02-28 21:43:43.888 | INFO     | src.docs:init_vector_store:27 - Populating Vector Store
2024-02-28 21:43:43.889 | INFO     | src.docs:init_vector_store:28 - Persisted @ :: ./persisted_data
2024-02-28 21:44:03.036 | SUCCESS  | src.docs:init_vector_store:32 - Done


In [13]:
easy_neg = finder.find_easy_negs_for(data_instance)

In [14]:
easy_neg

"Religionen_in_Deutschland\n\n=== Baptisten ===\nOrganisiert sind die autonomen Baptistengemeinden in Deutschland im Bund Evangelisch-Freikirchlicher Gemeinden. Daneben gibt es sogenannte ''Freie Baptistengemeinden'', die keinem übergeordneten Bund angehören und lediglich lockeren Kontakt untereinander pflegen. Dazu gehören unter anderem die Reformierten Baptisten, die Bibel-Baptisten, sowie eine größere Anzahl von Baptistengemeinden mit russlanddeutschem Hintergrund.\nDie Baptisten existieren seit 1834 in Deutschland. Der Baptismus verbreitete sich in Deutschland und Kontinentaleuropa vor allem durch den aus Varel stammenden Kaufmann und späteren Baptistenprediger Johann Gerhard Oncken, der in einer methodistischen Gemeinde in England seine Bekehrung erlebt hatte.\nSeit der Öffnung des Eisernen Vorhangs wanderten viele Deutsche aus den Staaten der ehemaligen Sowjetunion nach Deutschland ein (Russlanddeutsche, Kasachstandeutsche, Kirgisistandeutsche). Zu einem größeren Teil waren sie E

In [15]:
data_instance["contexts"].append(easy_neg)

In [20]:
prepare_jsonl_from_data_instance(data_instance)

[{'context': 'Gott\n\n=== Demografie ===\nEine Zusammenfassung von Umfrageergebnissen aus verschiedenen Staaten ergab im Jahr 2007, dass es weltweit zwischen 505 und 749 Millionen Atheisten und Agnostiker gibt. Laut der Encyclopædia Britannica gab es 2009 weltweit 640 Mio. Nichtreligiöse und Agnostiker (9,4 %), und weitere 139 Mio. Atheisten (2,0 %), hauptsächlich in der Volksrepublik China.\nBei einer Eurobarometer-Umfrage im Jahr 2005 wurde festgestellt, dass 52 % der damaligen EU-Bevölkerung glaubt, dass es einen Gott gibt. Eine vagere Frage nach dem Glauben an „eine andere spirituelle Kraft oder Lebenskraft“ wurde von weiteren 27 % positiv beantwortet. Bezüglich der Gottgläubigkeit bestanden große Unterschiede zwischen den einzelnen europäischen Staaten. Die Umfrage ergab, dass der Glaube an Gott in Staaten mit starkem kirchlichen Einfluss am stärksten verbreitet ist, dass mehr Frauen (58 %) als Männer (45 %) an einen Gott glauben und dass der Gottglaube mit höherem Alter, geringer

In [31]:
from typing import Any, List

def prepare_entire_dataset(dataset: Any, finder: EasyNegativeFinder) -> List[DataEntry]:
    prepared_dataset = list()

    for _, data_instance in tqdm(enumerate(dataset), total=len(dataset)):
        easy_neg = finder.find_easy_negs_for(data_instance)
        data_instance["contexts"].append(easy_neg)
        
        new_entries = prepare_jsonl_from_data_instance(data_instance)
        prepared_dataset.extend(new_entries)
        
    return prepared_dataset

In [32]:
prepared_dataset = prepare_entire_dataset(dataset, finder)

  0%|          | 0/3362 [00:00<?, ?it/s]

In [28]:
prepared_dataset[:5]

[{'context': 'Gott\n\n=== Demografie ===\nEine Zusammenfassung von Umfrageergebnissen aus verschiedenen Staaten ergab im Jahr 2007, dass es weltweit zwischen 505 und 749 Millionen Atheisten und Agnostiker gibt. Laut der Encyclopædia Britannica gab es 2009 weltweit 640 Mio. Nichtreligiöse und Agnostiker (9,4 %), und weitere 139 Mio. Atheisten (2,0 %), hauptsächlich in der Volksrepublik China.\nBei einer Eurobarometer-Umfrage im Jahr 2005 wurde festgestellt, dass 52 % der damaligen EU-Bevölkerung glaubt, dass es einen Gott gibt. Eine vagere Frage nach dem Glauben an „eine andere spirituelle Kraft oder Lebenskraft“ wurde von weiteren 27 % positiv beantwortet. Bezüglich der Gottgläubigkeit bestanden große Unterschiede zwischen den einzelnen europäischen Staaten. Die Umfrage ergab, dass der Glaube an Gott in Staaten mit starkem kirchlichen Einfluss am stärksten verbreitet ist, dass mehr Frauen (58 %) als Männer (45 %) an einen Gott glauben und dass der Gottglaube mit höherem Alter, geringer